In [61]:
import pandas as pd
import random
from sklearn.metrics import roc_auc_score

In [62]:
df = pd.read_csv("./kaggle-dataset/train_features.csv", index_col='id')
out = pd.read_csv("./kaggle-dataset/train_labels.csv", index_col='id')
df["charged_off"] = out["charged_off"].astype(int)
mode = [df["charged_off"].mode()[0],] * len(df)
print(roc_auc_score(df["charged_off"], mode))

0.5


In [63]:
months = {
    "Jan": 0,
    "Feb": 1,
    "Mar": 2,
    "Apr": 3,
    "May": 4,
    "Jun": 5,
    "Jul": 6,
    "Aug": 7,
    "Sep": 8,
    "Oct": 9,
    "Nov": 10,
    "Dec": 11
}
maps = {}

df = df.drop(columns=[
    "funded_amnt",
    "title"
])
df = df.dropna(axis=1, how='all')

df["term"] = df["term"].apply(lambda x: x[1:3]).astype(int)
df["int_rate"] = df["int_rate"].apply(lambda x: x[:-1]).astype(float)
df["zip_code"] = df["zip_code"].apply(lambda x: x[:-2]).astype(int)
df["earliest_cr_line"] = df["earliest_cr_line"].apply(lambda x: months[x.split("-")[0]]+int(x.split("-")[1])*12).astype(int)

df_train = df.sample(frac=0.8, random_state=1)
df_test  = df[~df.index.isin(df_train.index)]

for c in df_train:
    if df_train[c].dtype == "object":
        maps[c] = {}
        cats = list(df_train[c].astype('category').cat.categories.values)
        for cat in cats:
            maps[c][str(cat)] = random.uniform(0.0, 1.0)
        maps[c][None] = random.uniform(0.0, 1.0)
        df_train[c] = df_train[c].apply(lambda x: maps[c].get(str(x), maps[c][None])).astype(float)

for c in df_test:
    if df_test[c].dtype == "object":
        df_test[c] = df_test[c].apply(lambda x: maps[c].get(str(x), maps[c][None])).astype(float)

col_rep_dict = {}        
mean_nan_cols = [
    "dti",
    "annual_inc_joint",
    "dti_joint",
    "il_util",
    "all_util",
    "avg_cur_bal",
    "bc_open_to_buy",
    "bc_util",
    "percent_bc_gt_75",
    "revol_bal_joint"
]
for c in mean_nan_cols:
    col_rep_dict[c] = df_train[c].mean()
    df_train[c] = df_train[c].fillna(col_rep_dict[c])

min_nan_cols = [
    "num_tl_120dpd_2m",
    "pct_tl_nvr_dlq",
    "sec_app_inq_last_6mths",
    "sec_app_mort_acc",
    "sec_app_open_acc",
    "sec_app_revol_util",
    "sec_app_open_act_il",
    "sec_app_num_rev_accts",
    "sec_app_chargeoff_within_12_mths",
    "sec_app_collections_12_mths_ex_med",
    "sec_app_mths_since_last_major_derog"
]
for c in min_nan_cols:
    col_rep_dict[c] = df_train[c].min() - 1
    df_train[c] = df_train[c].fillna(col_rep_dict[c])

max_nan_cols = [
    "mths_since_last_delinq",
    "mths_since_last_record",
    "mths_since_last_major_derog",
    "mths_since_rcnt_il",
    "mo_sin_old_il_acct",
    "mths_since_recent_bc",
    "mths_since_recent_bc_dlq",
    "mths_since_recent_inq",
    "mths_since_recent_revol_delinq"
]
for c in max_nan_cols:
    col_rep_dict[c] = 1 + df_train[c].max()
    df_train[c] = df_train[c].fillna(col_rep_dict[c])
    
for c in col_rep_dict:
    df_test[c] = df_test[c].fillna(col_rep_dict[c])

C:\Users\brit2\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\brit2\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [64]:
df_train.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,charged_off
id,,,,,,,,,,,,,,,,,,,,,
20726,10000,36,15.04,346.85,0.023452,0.658223,0.276700,0.056923,0.288044,145000.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,1
35481,35000,60,18.45,897.37,0.453628,0.818888,0.389756,0.810797,0.042758,85000.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,1
39942,2000,36,14.07,68.43,0.023452,0.903010,0.668601,0.080186,0.288044,30604.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,1
36509,35000,60,10.08,745.03,0.869217,0.411706,0.497005,0.056923,0.288044,72000.0,...,5.0,17.0,7.1,3.0,22.0,0.0,0.0,-1.0,0.909097,0
27939,7000,36,12.61,234.55,0.023452,0.954845,0.592261,0.872037,0.288044,95000.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,1


In [65]:
df_test.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,charged_off
id,,,,,,,,,,,,,,,,,,,,,
24414,6300,36,14.07,215.54,0.023452,0.903010,0.668601,0.401192,0.042758,39000.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,0
16701,15600,36,6.72,479.69,0.315397,0.787259,0.046961,0.056923,0.288044,80000.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,0
11504,35000,36,5.31,1053.86,0.315397,0.267025,0.496981,0.056923,0.288044,190000.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,0
19667,40000,36,6.71,1229.79,0.315397,0.787259,0.524609,0.292481,0.288044,327843.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,0
39343,18000,60,12.62,406.07,0.023452,0.954845,0.031576,0.872037,0.288044,142000.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.909097,0


In [66]:
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [67]:
X_train = df_train[[c for c in df_train if c != "charged_off"]]
y_train = df_train["charged_off"]

X_test = df_test[[c for c in df_test if c != "charged_off"]]
y_test = df_test["charged_off"]

for i in range(1,10+1):
    probs = XGBClassifier(max_depth=i).fit(X_train, y_train).predict_proba(X_test)[:,1]
    print(i, "\t", roc_auc_score(y_test, probs))

1 	 0.7282947228491906
2 	 0.7474064278493984
3 	 0.7535002838666796
4 	 0.7573211986097456
5 	 0.758251727432599
6 	 0.7583517731282107
7 	 0.758269936441558
8 	 0.755868701275323
9 	 0.7492217898832684
10 	 0.7542931027320436


In [71]:
X = pd.read_csv("./kaggle-dataset/test_features.csv", index_col='id')

months = {
    "Jan": 0,
    "Feb": 1,
    "Mar": 2,
    "Apr": 3,
    "May": 4,
    "Jun": 5,
    "Jul": 6,
    "Aug": 7,
    "Sep": 8,
    "Oct": 9,
    "Nov": 10,
    "Dec": 11
}

X = X.drop(columns=[
    "funded_amnt",
    "title"
])
X = X.dropna(axis=1, how='all')

X["term"] = X["term"].apply(lambda x: x[1:3]).astype(int)
X["int_rate"] = X["int_rate"].apply(lambda x: x[:-1]).astype(float)
X["zip_code"] = X["zip_code"].apply(lambda x: x[:-2]).astype(int)
X["earliest_cr_line"] = X["earliest_cr_line"].apply(lambda x: months[x.split("-")[0]]+int(x.split("-")[1])*12).astype(int)

for c in X:
    if X[c].dtype == "object":
        X[c] = X[c].apply(lambda x: maps[c].get(str(x), maps[c][None])).astype(float)
for c in col_rep_dict:
    X[c] = X[c].fillna(col_rep_dict[c])

X["charged_off"] = XGBClassifier(max_depth=7).fit(X_train, y_train).predict_proba(X)[:,1]
out = X[["charged_off"]]
out.to_csv("./kaggle-dataset/submission_1.csv", )